In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_train_1 = pd.read_csv('/home/dsc/Downloads/CMAPSSData/train_FD001.txt', sep=" ", header=None)

list_units_1 = 3
operational_list = []
for n in range(1,list_units_1+1):
    operational = "operarional_setting_" + str(n)
    operational_list.append(operational)

list_units_2 = 23
sensor_measurement_list = []
for n in range(1,list_units_2+1):
    sensor = "sensor_measurement_" + str(n)
    sensor_measurement_list.append(sensor)
df_train_1.columns = ["ID", "Cycle"] + operational_list + sensor_measurement_list

df_train_1 = df_train_1.dropna(axis=1)

max_cycle_dataset_1 = df_train_1.groupby(["ID"])["Cycle"].max().reset_index()
max_cycle_dataset_1 = max_cycle_dataset_1.rename(columns={"Cycle":"Max_cycle"})
df_train_1 = df_train_1.merge(max_cycle_dataset_1, how="inner", on="ID")
df_train_1["RUL"] = df_train_1["Max_cycle"] - df_train_1["Cycle"]
df_train_1 = df_train_1.drop(["Max_cycle"],axis=1)

df_train_1.drop(columns="operarional_setting_3",axis=1, inplace=True)
columns_to_delete = ['sensor_measurement_1', 'sensor_measurement_5', 'sensor_measurement_6', 'sensor_measurement_10', 'sensor_measurement_16', 'sensor_measurement_18', 'sensor_measurement_19']
df_train_1.drop(columns=columns_to_delete, axis=1, inplace=True)

#df_train_1.drop(columns="sensor_measurement_14", axis=1, inplace=True)

df_train_1.shape

(20631, 19)

In [2]:
df_test_1 = pd.read_csv('/home/dsc/Downloads/CMAPSSData/test_FD001.txt', sep=" ", header=None)
list_units_1 = 3
operational_list = []
for n in range(1,list_units_1+1):
    operational = "operarional_setting_" + str(n)
    operational_list.append(operational)
list_units_2 = 23
sensor_measurement_list = []
for n in range(1,list_units_2+1):
    sensor = "sensor_measurement_" + str(n)
    sensor_measurement_list.append(sensor)

df_test_1.columns = ["ID", "Cycle"] + operational_list + sensor_measurement_list
df_test_1 = df_test_1.dropna(axis=1)
df_test_1.drop(columns="operarional_setting_3",axis=1, inplace=True)
df_test_1.drop(columns=columns_to_delete, axis=1, inplace=True)
#df_test_1.drop(columns="sensor_measurement_14", axis=1, inplace=True)

df_test_1.shape

(13096, 18)

In [3]:
df_rul_1 = pd.read_csv('/home/dsc/Downloads/CMAPSSData/RUL_FD001.txt', sep=" ", header=None)


df_rul_1.dropna(axis=1,inplace=True)
#df_rul_1["ID"] = np.arange(1, 101)
df_rul_1.columns = ["RUL"]
#df_rul_1 = df_rul_1[["ID","RUL"]]
df_rul_1.shape

(100, 1)

In [4]:
labels_to_drop = ["ID","Cycle"]
X_train1 = df_train_1.drop(labels_to_drop+["RUL"], axis=1)
y_train1 = df_train_1["RUL"]

X_test1 = df_test_1.groupby("ID").last().reset_index().drop(labels_to_drop, axis=1)
y_test1 = df_rul_1
print("X_train columns: ", X_train1.columns, "\n",
     "X_test columns: ", X_test1.columns, "\n",
     y_train1.shape)

X_train columns:  Index(['operarional_setting_1', 'operarional_setting_2',
       'sensor_measurement_2', 'sensor_measurement_3', 'sensor_measurement_4',
       'sensor_measurement_7', 'sensor_measurement_8', 'sensor_measurement_9',
       'sensor_measurement_11', 'sensor_measurement_12',
       'sensor_measurement_13', 'sensor_measurement_14',
       'sensor_measurement_15', 'sensor_measurement_17',
       'sensor_measurement_20', 'sensor_measurement_21'],
      dtype='object') 
 X_test columns:  Index(['operarional_setting_1', 'operarional_setting_2',
       'sensor_measurement_2', 'sensor_measurement_3', 'sensor_measurement_4',
       'sensor_measurement_7', 'sensor_measurement_8', 'sensor_measurement_9',
       'sensor_measurement_11', 'sensor_measurement_12',
       'sensor_measurement_13', 'sensor_measurement_14',
       'sensor_measurement_15', 'sensor_measurement_17',
       'sensor_measurement_20', 'sensor_measurement_21'],
      dtype='object') 
 (20631,)


In [5]:
X_train1.shape, y_train1.shape, X_test1.shape, df_rul_1.shape

((20631, 16), (20631,), (100, 16), (100, 1))

In [6]:
df_rul_1

,RUL
0,112
1,98
2,69
3,82
4,91
...,...
95,137
96,82
97,59
98,117


#                                              MODELS AS IT IS -------------------------------------------------------------------------

## Linear Regression

In [7]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train1, y_train1)

LinearRegression()

In [8]:
from sklearn.metrics import mean_absolute_error, r2_score
preds_train1 = lr.predict(X_train1)
print("TRAIN", "\n",
      "MAE: ",mean_absolute_error(preds_train1, y_train1), "\n",
      "R2 score: ", r2_score(y_train1, preds_train1))

TRAIN 
 MAE:  34.11381828711524 
 R2 score:  0.5794932912981662


In [9]:
preds_test1 = lr.predict(X_test1)
print("TRAIN", "\n",
      "MAE: ",mean_absolute_error(preds_test1, df_rul_1), "\n",
      "R2 score: ", r2_score(df_rul_1, preds_test1))

TRAIN 
 MAE:  25.593806249895135 
 R2 score:  0.4054344040525887


#### TRAIN = 0.579
#### TEST = 0.405

## RANDOM FOREST REGRESSOR

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth}
print(random_grid)

{'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]}


In [11]:
regr = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=regr, param_distributions=random_grid, random_state=42)
rf_random.fit(X_train1, y_train1)

RandomizedSearchCV(estimator=RandomForestRegressor(),
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]},
                   random_state=42)

In [12]:
rf_random.best_params_

{'n_estimators': 50, 'max_depth': 10}

In [13]:
n_estimators = list(rf_random.best_params_.values())[0]
max_depth = list(rf_random.best_params_.values())[1]

In [14]:
hyper_regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
hyper_regr.fit(X_train1, y_train1)

RandomForestRegressor(max_depth=10, n_estimators=50)

In [15]:
rul_train_pred_rf = hyper_regr.predict(X_train1)
print("RANDOM FOREST TRAIN SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(y_train1, rul_train_pred_rf), "\n",
      "R2 score: ", r2_score(y_train1, rul_train_pred_rf))

RANDOM FOREST TRAIN SCORES - - - -  
 MAE:  25.448241817037026 
 R2 score:  0.7212645194371912


In [16]:
rul_test_pred_rf = hyper_regr.predict(X_test1)
print("RANDOM FOREST TEST SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(df_rul_1, rul_test_pred_rf), "\n",
      "R2 score: ", r2_score(df_rul_1, rul_test_pred_rf))

RANDOM FOREST TEST SCORES - - - -  
 MAE:  24.213833455593882 
 R2 score:  0.3938775818528696


#### TRAIN = 0.721
#### TEST = 0.387

## SVR

In [17]:
X_train1 = df_train_1.drop(labels_to_drop+["RUL"], axis=1)
y_train1 = df_train_1["RUL"]

X_test1 = df_test_1.groupby("ID").last().reset_index().drop(labels_to_drop, axis=1)
y_test1 = df_rul_1

In [18]:
from sklearn.svm import SVR

svr = SVR(kernel="linear")
svr.fit(X_train1, y_train1)
preds_train_svr = svr.predict(X_train1)

In [19]:
preds_train_svr = svr.predict(X_train1)
print("SVR TRAIN SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(y_train1, preds_train_svr), "\n",
      "R2 score: ", r2_score(y_train1, preds_train_svr))

SVR TRAIN SCORES - - - -  
 MAE:  41.08592612483098 
 R2 score:  0.4277772082827639


In [20]:
preds_test_svr = svr.predict(X_test1)
print("SVR TEST SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(preds_test_svr, y_test1), "\n",
      "R2 score: ", r2_score(preds_test_svr, y_test1))

SVR TEST SCORES - - - -  
 MAE:  35.12016117707506 
 R2 score:  0.2231324732734803


#### TRAIN = 0.4277
#### TEST = 0.2231

# MinMaxScaler

In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train1 = df_train_1.drop(labels_to_drop+["RUL"], axis=1)
y_train1 = df_train_1["RUL"]

X_test1 = df_test_1.groupby("ID").last().reset_index().drop(labels_to_drop, axis=1)
y_test1 = df_rul_1

scaler.fit(X_train1)
X_train_scal = scaler.transform(X_train1)
X_test_scal = scaler.transform(X_test1)

## Linear Regression (+ minMaxScal)

In [22]:
lr = LinearRegression()
lr.fit(X_train_scal, y_train1)

LinearRegression()

In [23]:
preds_train1 = lr.predict(X_train_scal)
print("TRAIN", "\n",
      "MAE: ",mean_absolute_error(y_train1, preds_train1), "\n",
      "R2 score: ", r2_score(y_train1, preds_train1))

TRAIN 
 MAE:  34.113818287114796 
 R2 score:  0.5794932912981678


In [24]:
preds_test1 = lr.predict(X_test_scal)
print("TRAIN", "\n",
      "MAE: ",mean_absolute_error(y_test1, preds_test1), "\n",
      "R2 score: ", r2_score(y_test1, preds_test1))

TRAIN 
 MAE:  25.593806249892133 
 R2 score:  0.40543440405265174


#### TRAIN = 0.579
#### TEST = 0.405

## Random Forest Regressor (+ minMaxScal)

In [25]:
scaler = MinMaxScaler()

X_train1 = df_train_1.drop(labels_to_drop+["RUL"], axis=1)
y_train1 = df_train_1["RUL"]

X_test1 = df_test_1.groupby("ID").last().reset_index().drop(labels_to_drop, axis=1)
y_test1 = df_rul_1

scaler.fit(X_train1)
X_train_scal = scaler.transform(X_train1)
X_test_scal = scaler.transform(X_test1)

In [26]:
regr = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=regr, param_distributions=random_grid, random_state=42)
rf_random.fit(X_train_scal, y_train1)

KeyboardInterrupt: 

In [ ]:
n_estimators = list(rf_random.best_params_.values())[0]
max_depth = list(rf_random.best_params_.values())[1]

In [ ]:
hyper_regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
hyper_regr.fit(X_train_scal, y_train1)

In [ ]:
rul_train_pred_rf = hyper_regr.predict(X_train_scal)
print("RANDOM FOREST TRAIN SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(y_train1, rul_train_pred_rf), "\n",
      "R2 score: ", r2_score(y_train1, rul_train_pred_rf))

In [ ]:
rul_test_pred_rf = hyper_regr.predict(X_test_scal)
print("RANDOM FOREST TEST SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(df_rul_1, rul_test_pred_rf), "\n",
      "R2 score: ", r2_score(df_rul_1, rul_test_pred_rf))

#### TRAIN = 0.579
#### TEST = 0.405

## SVR (+ minMaxScal)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train1 = df_train_1.drop(labels_to_drop+["RUL"], axis=1)
y_train1 = df_train_1["RUL"]

X_test1 = df_test_1.groupby("ID").last().reset_index().drop(labels_to_drop, axis=1)
y_test1 = df_rul_1

scaler.fit(X_train1)
X_train_scal = scaler.transform(X_train1)
X_test_scal = scaler.transform(X_test1)

In [ ]:
svr = SVR(kernel="linear")
svr.fit(X_train_scal, y_train1)
preds_train_svr = svr.predict(X_train_scal)

In [ ]:
preds_train_svr = svr.predict(X_train_scal)
print("SVR + minMaxScal TRAIN SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(y_train1, preds_train_svr), "\n",
      "R2 score: ", r2_score(y_train1, preds_train_svr))

In [ ]:
preds_test_svr = svr.predict(X_test_scal)
print("SVR + minMaxScal TEST SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(y_test1, preds_test_svr), "\n",
      "R2 score: ", r2_score(y_test1, preds_test_svr))

#### TRAIN = 0.5609
#### TEST = 0.5984

# TOP FEATURES

In [ ]:
df_1 = df_train_1[df_train_1["ID"]==1]
x = df_1["Cycle"].values.reshape(-1,1)

cols = df_1.columns[2:-1]
fig, axes = plt.subplots(len(cols), 1, figsize=(10,35))
slopes = []
for col, ax in zip(cols, axes):
    linear = LinearRegression()
    y = df_1[col].values.reshape(-1,1)
    linear.fit(x,y)
    preds = linear.predict(x)
    ax.plot(x,y, label=col)
    ax.plot(x,preds,"g",label='trend')
    ax.legend(loc=2)
    slope = linear.coef_
    slopes.append(slope)

In [ ]:
slopes_df = pd.DataFrame({"columns": cols, "slope":slopes})
slopes_df["slope"] = slopes_df["slope"].apply(lambda x: abs(x))
slopes_df = slopes_df.sort_values("slope", ascending=False)

## Linear Regression (+TopFeatures)

In [ ]:
columns = slopes_df["columns"].values.tolist()
column=[]
train_scores = []
test_scores = []
for col in columns:
    column.append(col)
    
    df = df_train_1[["ID","Cycle"]+column]
    X_train = df.drop(columns=["ID","Cycle"], axis=1)
    
    y_train = df_train_1["RUL"]
    df_test = df_test_1[["ID", "Cycle"]+column]
    df_test_red = df_test.groupby("ID").last().reset_index()
    X_test = df_test_red.drop(columns=["ID","Cycle"], axis=1)
    y_test = df_rul_1
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    preds_train = lr.predict(X_train)
    r2_tr = r2_score(y_train, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} top features: ".format(len(column)), r2_score(y_train, preds_train))
    
    preds_test = lr.predict(X_test)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} top features: ".format(len(column)), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("LINEAR REGRESSION Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

## RanfdomForestRegressor (+TopFeatures)

In [ ]:
columns = slopes_df["columns"].values.tolist()
column=[]
train_scores = []
test_scores = []
for col in columns:
    column.append(col)
    
    df = df_train_1[["ID","Cycle"]+column]
    X_train = df.drop(columns=["ID","Cycle"], axis=1)
    y_train = df_train_1["RUL"]
    df_test = df_test_1[["ID", "Cycle"]+column]
    df_test_red = df_test.groupby("ID").last().reset_index()
    X_test = df_test_red.drop(columns=["ID","Cycle"], axis=1)
    y_test = df_rul_1
    
    
    regr = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator=regr, param_distributions=random_grid, random_state=42)
    rf_random.fit(X_train, y_train1)
    
    n_estimators = list(rf_random.best_params_.values())[0]
    max_depth = list(rf_random.best_params_.values())[1]
    
    hyper_regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    hyper_regr.fit(X_train, y_train)
    
    rul_train_pred_rf = hyper_regr.predict(X_train)
    r2_tr = r2_score(y_train, rul_train_pred_rf)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} top features: ".format(len(column)), r2_score(y_train, rul_train_pred_rf))
    
    rul_test_pred_rf = hyper_regr.predict(X_test)
    r2_ts = r2_score(y_test, rul_test_pred_rf)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} top features: ".format(len(column)), r2_score(y_test, rul_test_pred_rf))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("RANDOMFOREST REGRESSOR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

## SVR (+TopFeatures)

In [ ]:
columns = slopes_df["columns"].values.tolist()
column=[]
train_scores = []
test_scores = []
for col in columns:
    column.append(col)
    
    df = df_train_1[["ID","Cycle"]+column]
    X_train = df.drop(columns=["ID","Cycle"], axis=1)
    
    y_train = df_train_1["RUL"]
    df_test = df_test_1[["ID", "Cycle"]+column]
    df_test_red = df_test.groupby("ID").last().reset_index()
    X_test = df_test_red.drop(columns=["ID","Cycle"], axis=1)
    y_test = df_rul_1
    
    svr = SVR(kernel="linear")
    svr.fit(X_train, y_train)
    preds_train = svr.predict(X_train)
    r2_tr = r2_score(y_train, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} top features: ".format(len(column)), r2_score(y_train, preds_train))
    
    preds_test = svr.predict(X_test)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} top features: ".format(len(column)), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

# y_CLIP

## LinearRegression (+y_clip)

In [ ]:
clips = np.arange(100,170,5).tolist()
train_scores = []
test_scores = []
for clip in clips:
    X_train = df_train_1.drop(columns=["ID","Cycle","RUL","operarional_setting_1","operarional_setting_2"], axis=1)
    y_train_clip = df_train_1["RUL"].clip(upper=clip)

    df_test_1_red = df_test_1.groupby("ID").last().reset_index()
    X_test = df_test_1_red.drop(columns=["ID","Cycle","operarional_setting_1","operarional_setting_2"], axis=1)
    y_test = df_rul_1

    lr = LinearRegression()
    lr.fit(X_train, y_train_clip)
    preds_train = lr.predict(X_train)
    r2_tr = r2_score(y_train_clip, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_train_clip, preds_train))
    
    preds_test = lr.predict(X_test)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

## RANDOM FOREST (+clip)

In [ ]:
clips = np.arange(100,170,10).tolist()
train_scores_rf = []
test_scores_rf = []
for clip in clips:
    X_train = df_train_1.drop(columns=["ID","Cycle","RUL","operarional_setting_1","operarional_setting_2"], axis=1)
    y_train_clip = df_train_1["RUL"].clip(upper=clip)

    df_test_1_red = df_test_1.groupby("ID").last().reset_index()
    X_test = df_test_1_red.drop(columns=["ID","Cycle","operarional_setting_1","operarional_setting_2"], axis=1)
    y_test = df_rul_1

    regr = RandomForestRegressor()
    rf_random_clip = RandomizedSearchCV(estimator=regr, param_distributions=random_grid, random_state=42)
    rf_random_clip.fit(X_train, y_train_clip)
    n_estimators = list(rf_random_clip.best_params_.values())[0]
    max_depth = list(rf_random_clip.best_params_.values())[1]
    hyper_regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    hyper_regr.fit(X_train, y_train_clip)
    rul_train_pred_rf = hyper_regr.predict(X_train)
    r2_tr = r2_score(y_train_clip, rul_train_pred_rf)
    train_scores_rf.append(r2_tr)
    print(" TRAIN SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_train_clip, rul_train_pred_rf))
    
    rul_test_pred_rf = hyper_regr.predict(X_test)
    r2_ts = r2_score(y_test, rul_test_pred_rf)
    test_scores_rf.append(r2_ts)
    print(" TEST SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_test, rul_test_pred_rf))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

## SVR (+y_CLIP)

In [ ]:
clips = np.arange(100,170,5).tolist()
train_scores = []
test_scores = []
for clip in clips:
    X_train = df_train_1.drop(columns=["ID","Cycle","RUL","operarional_setting_1","operarional_setting_2"], axis=1)
    y_train_clip = df_train_1["RUL"].clip(upper=clip)

    df_test_1_red = df_test_1.groupby("ID").last().reset_index()
    X_test = df_test_1_red.drop(columns=["ID","Cycle","operarional_setting_1","operarional_setting_2"], axis=1)
    y_test = df_rul_1

    svr = SVR(kernel="linear")
    svr.fit(X_train, y_train_clip)
    preds_train = svr.predict(X_train)
    r2_tr = r2_score(y_train_clip, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_train_clip, preds_train))
    
    preds_test = svr.predict(X_test)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

# MIN_MAX_SCALER + TOP_FEATURES

## LinearRegression (+MinMax + Top_feat)

In [ ]:
columns = slopes_df["columns"].values.tolist()
column=[]
train_scores = []
test_scores = []
for col in columns:
    column.append(col)
    
    df = df_train_1[["ID","Cycle"]+column]
    X_train = df.drop(columns=["ID","Cycle"], axis=1)
    
    y_train = df_train_1["RUL"]
    df_test = df_test_1[["ID", "Cycle"]+column]
    df_test_red = df_test.groupby("ID").last().reset_index()
    X_test = df_test_red.drop(columns=["ID","Cycle"], axis=1)
    y_test = df_rul_1
    
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scal = scaler.transform(X_train)
    X_test_scal = scaler.transform(X_test)
    
    lr = LinearRegression()
    lr.fit(X_train_scal, y_train)
    preds_train = lr.predict(X_train_scal)
    r2_tr = r2_score(y_train, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} top features: ".format(len(column)), r2_score(y_train, preds_train))
    
    preds_test = lr.predict(X_test_scal)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} top features: ".format(len(column)), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

## RanfomForest (+MinMax + Top_feat)

In [ ]:
columns = slopes_df["columns"].values.tolist()
column=[]
train_scores = []
test_scores = []
for col in columns:
    column.append(col)
    
    df = df_train_1[["ID","Cycle"]+column]
    X_train = df.drop(columns=["ID","Cycle"], axis=1)
    y_train = df_train_1["RUL"]
    df_test = df_test_1[["ID", "Cycle"]+column]
    df_test_red = df_test.groupby("ID").last().reset_index()
    X_test = df_test_red.drop(columns=["ID","Cycle"], axis=1)
    y_test = df_rul_1
    
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scal = scaler.transform(X_train)
    X_test_scal = scaler.transform(X_test)
    
    regr = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator=regr, param_distributions=random_grid, random_state=42)
    rf_random.fit(X_train_scal, y_train)
    
    n_estimators = list(rf_random.best_params_.values())[0]
    max_depth = list(rf_random.best_params_.values())[1]
    
    hyper_regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    hyper_regr.fit(X_train_scal, y_train)
    
    rul_train_pred_rf = hyper_regr.predict(X_train_scal)
    r2_tr = r2_score(y_train, rul_train_pred_rf)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} top features: ".format(len(column)), r2_score(y_train, rul_train_pred_rf))
    
    rul_test_pred_rf = hyper_regr.predict(X_test_scal)
    r2_ts = r2_score(y_test, rul_test_pred_rf)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} top features: ".format(len(column)), r2_score(y_test, rul_test_pred_rf))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

## SVR (+MinMax + top_feat)

In [ ]:
columns = slopes_df["columns"].values.tolist()
column=[]
train_scores = []
test_scores = []
for col in columns:
    column.append(col)
    
    df = df_train_1[["ID","Cycle"]+column]
    X_train = df.drop(columns=["ID","Cycle"], axis=1)
    
    y_train = df_train_1["RUL"]
    df_test = df_test_1[["ID", "Cycle"]+column]
    df_test_red = df_test.groupby("ID").last().reset_index()
    X_test = df_test_red.drop(columns=["ID","Cycle"], axis=1)
    y_test = df_rul_1
    
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scal = scaler.transform(X_train)
    X_test_scal = scaler.transform(X_test)
    
    svr = SVR(kernel="linear")
    svr.fit(X_train_scal, y_train)
    preds_train = svr.predict(X_train_scal)
    r2_tr = r2_score(y_train, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} top features: ".format(len(column)), r2_score(y_train, preds_train))
    
    preds_test = svr.predict(X_test_scal)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} top features: ".format(len(column)), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

# MIN_MAX_SCALER + y_Clip

## Linear Regression (+minMax + y_clip)

In [ ]:
clips = np.arange(100,170,5).tolist()
train_scores = []
test_scores = []
for clip in clips:
    X_train = df_train_1.drop(columns=["ID","Cycle","RUL","operarional_setting_1","operarional_setting_2"], axis=1)
    y_train_clip = df_train_1["RUL"].clip(upper=clip)

    df_test_1_red = df_test_1.groupby("ID").last().reset_index()
    X_test = df_test_1_red.drop(columns=["ID","Cycle","operarional_setting_1","operarional_setting_2"], axis=1)
    y_test = df_rul_1

    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scal = scaler.transform(X_train)
    X_test_scal = scaler.transform(X_test)
    
    lr = LinearRegression()
    lr.fit(X_train_scal, y_train_clip)
    preds_train = lr.predict(X_train_scal)
    r2_tr = r2_score(y_train_clip, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_train_clip, preds_train))
    
    preds_test = lr.predict(X_test_scal)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

## RandomForest (+minMax + y_clip)

In [ ]:
clips = np.arange(100,170,10).tolist()
train_scores_rf = []
test_scores_rf = []
for clip in clips:
    X_train = df_train_1.drop(columns=["ID","Cycle","RUL","operarional_setting_1","operarional_setting_2"], axis=1)
    y_train_clip = df_train_1["RUL"].clip(upper=clip)

    df_test_1_red = df_test_1.groupby("ID").last().reset_index()
    X_test = df_test_1_red.drop(columns=["ID","Cycle","operarional_setting_1","operarional_setting_2"], axis=1)
    y_test = df_rul_1

    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scal = scaler.transform(X_train)
    X_test_scal = scaler.transform(X_test)
    
    regr = RandomForestRegressor()
    rf_random_clip = RandomizedSearchCV(estimator=regr, param_distributions=random_grid, random_state=42)
    rf_random_clip.fit(X_train_scal, y_train_clip)
    n_estimators = list(rf_random_clip.best_params_.values())[0]
    max_depth = list(rf_random_clip.best_params_.values())[1]
    hyper_regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    hyper_regr.fit(X_train_scal, y_train_clip)
    
    rul_train_pred_rf = hyper_regr.predict(X_train_scal)
    r2_tr = r2_score(y_train_clip, rul_train_pred_rf)
    train_scores_rf.append(r2_tr)
    print(" TRAIN SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_train_clip, rul_train_pred_rf))
    
    rul_test_pred_rf = hyper_regr.predict(X_test_scal)
    r2_ts = r2_score(y_test, rul_test_pred_rf)
    test_scores_rf.append(r2_ts)
    print(" TEST SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_test, rul_test_pred_rf))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

## SVR (+minMax + y_clip)

In [ ]:
clips = np.arange(100,170,5).tolist()
train_scores = []
test_scores = []
for clip in clips:
    X_train = df_train_1.drop(columns=["ID","Cycle","RUL","operarional_setting_1","operarional_setting_2"], axis=1)
    y_train_clip = df_train_1["RUL"].clip(upper=clip)

    df_test_1_red = df_test_1.groupby("ID").last().reset_index()
    X_test = df_test_1_red.drop(columns=["ID","Cycle","operarional_setting_1","operarional_setting_2"], axis=1)
    y_test = df_rul_1
    
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scal = scaler.transform(X_train)
    X_test_scal = scaler.transform(X_test)
    
    svr = SVR(kernel="linear")
    svr.fit(X_train_scal, y_train_clip)
    preds_train = svr.predict(X_train_scal)
    r2_tr = r2_score(y_train_clip, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_train_clip, preds_train))
    
    preds_test = svr.predict(X_test_scal)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(column)),train_scores,label="train scores")
plt.plot(range(0,len(column)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("SVR Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} features out of 15".format(mx_idx))

#### TRAIN = 0.5494
#### TEST = 0.4087

# TOP_FEATURES + y_clip

## LinearRegression (+top_feat + y_clip)

In [ ]:
columns = slopes_df["columns"].values.tolist()
column=[]
train_scores = []
test_scores = []
for col in columns:
    column.append(col)
    clips = np.arange(100,170,5).tolist()
    for clip in clips:
        df = df_train_1[["ID","Cycle"]+column]
        
        X_train = df.drop(columns=["ID","Cycle"], axis=1)
        y_train_clip = df_train_1["RUL"].clip(upper=clip)
        
        df_test = df_test_1[["ID", "Cycle"]+column]
        df_test_red = df_test.groupby("ID").last().reset_index()
        X_test = df_test_red.drop(columns=["ID","Cycle"], axis=1)
        y_test = df_rul_1


        lr = LinearRegression()
        lr.fit(X_train_scal, y_train)
        preds_train = lr.predict(X_train_scal)
        r2_tr = r2_score(y_train, preds_train)
        train_scores.append(r2_tr)
        print(" TRAIN SCORE WITH {} top features: ".format(len(column)), r2_score(y_train, preds_train))

        preds_test = lr.predict(X_test_scal)
        r2_ts = r2_score(y_test, preds_test)
        test_scores.append(r2_ts)
        print(" TEST SCORE WITH {} top features: ".format(len(column)), r2_score(y_test, preds_test))

In [ ]:
clips = np.arange(100,170,5).tolist()
train_scores = []
test_scores = []
for clip in clips:
    X_train = df_train_1.drop(columns=["ID","Cycle","RUL","operarional_setting_1","operarional_setting_2"], axis=1)
    y_train_clip = df_train_1["RUL"].clip(upper=clip)

    df_test_1_red = df_test_1.groupby("ID").last().reset_index()
    X_test = df_test_1_red.drop(columns=["ID","Cycle","operarional_setting_1","operarional_setting_2"], axis=1)
    y_test = df_rul_1
    
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scal = scaler.transform(X_train)
    X_test_scal = scaler.transform(X_test)
    
    svr = SVR(kernel="linear")
    svr.fit(X_train_scal, y_train_clip)
    preds_train = svr.predict(X_train_scal)
    r2_tr = r2_score(y_train_clip, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_train_clip, preds_train))
    
    preds_test = svr.predict(X_test_scal)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_test, preds_test))

In [ ]:
524658468-*251
ç485


No great changes by using MinMax Scaler

# Clip RUL

In [ ]:
X_train1 = df_train_1.drop(columns=["ID","Cycle","RUL"], axis=1)
y_train_1clip = df_train_1["RUL"].clip(upper=125)

df_test_1_red = df_test_1.groupby("ID").last().reset_index()
X_test1 = df_test_1_red.drop(columns=["ID","Cycle"], axis=1)
y_test = df_rul_1

lr = LinearRegression()
lr.fit(X_train1, y_train_1clip)

In [ ]:
preds_train_clip = lr.predict(X_train1)
print("TRAIN SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(y_train_1clip,preds_train_clip), "\n",
      "R2 score: ", r2_score(y_train_1clip, preds_train_clip))

In [ ]:
preds_test_clip = lr.predict(X_test1)
print("TRAIN SCORES - - - - ", "\n",
    "MAE: ", mean_absolute_error(y_test,preds_test_clip), "\n",
      "R2 score: ", r2_score(y_test, preds_test_clip))

In [ ]:
clips = np.arange(100,150,5).tolist()
train_scores = []
test_scores = []
for clip in clips:
    X_train = df_train_1.drop(columns=["ID","Cycle","RUL"], axis=1)
    y_train_clip = df_train_1["RUL"].clip(upper=clip)

    df_test_1_red = df_test_1.groupby("ID").last().reset_index()
    X_test = df_test_1_red.drop(columns=["ID","Cycle"], axis=1)
    y_test = df_rul_1

    lr = LinearRegression()
    lr.fit(X_train, y_train_clip)
    preds_train = lr.predict(X_train)
    r2_tr = r2_score(y_train_clip, preds_train)
    train_scores.append(r2_tr)
    print(" TRAIN SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_train_clip, preds_train))
    
    preds_test = lr.predict(X_test)
    r2_ts = r2_score(y_test, preds_test)
    test_scores.append(r2_ts)
    print(" TEST SCORE WITH {} upper clipped RUL: ".format(clip), r2_score(y_test, preds_test))

In [ ]:
plt.plot(range(0,len(clips)),train_scores,label="train scores")
plt.plot(range(0,len(clips)), test_scores, label="test scores")
plt.legend()
mx_test = max(test_scores)
mx_idx = test_scores.index(mx_test)
train_mx_test = train_scores[mx_idx]
print("Train Score from max Test Score: ", train_mx_test, "\n",
      "Max Test Score: ", mx_test, "\n",
     "Achieved with {} upper clipped RUL".format(clip))

## PCA with top slope features

In [ ]:
top_10_features_df = slopes_df.sort_values("slope", ascending=False).head(5)
top_10_features = top_10_features_df["columns"].values.tolist()
top_10_df = df_train_1[top_10_features]

In [ ]:
top_10_df.values

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()
pca.fit(top_10_df.values)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
sensors_pca = pca.transform()